# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import math
import random
import matplotlib.pyplot as plt
import seaborn as sns

# Plot styles
plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'
sns.set_style('darkgrid')
%matplotlib inline

from itertools import combinations

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.power import TTestIndPower, TTestPower
power_analysis = TTestIndPower()

import warnings
warnings.filterwarnings("ignore")

from flatiron_stats import *

In [2]:
#Your code here
df = pd.read_csv('homepage_actions.csv', encoding='latin-1')
print(len(df))
df.head()

8188


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
print(df.group.unique())
print(df.group.nunique())
print(df.group.value_counts(normalize=True))

['experiment' 'control']
2
control       0.520762
experiment    0.479238
Name: group, dtype: float64


In [4]:
print(df.action.unique())
print(df.action.nunique())
print(df.action.value_counts(normalize=True))
print(df.action.value_counts())

['view' 'click']
2
view     0.772838
click    0.227162
Name: action, dtype: float64
view     6328
click    1860
Name: action, dtype: int64


In [5]:
click_only_id = df[(df.action == 'click') & ~(df.action == 'view')].id
view_only_id = df[~(df.action == 'click') & (df.action == 'view')].id
click_view_id = df[(df.action == 'click') & (df.action == 'view')].id
print('click only:',len(click_only_id))
print('view only:',len(view_only_id))
print('click_view:',len(click_view_id))
print('total:',len(click_only_id)+len(view_only_id)-len(click_view_id))

click only: 1860
view only: 6328
click_view: 0
total: 8188


In [6]:
print('click only (repetition removed):',len(set(click_only_id)))
print('view only (repetition removed):',len(set(view_only_id)))

click only (repetition removed): 1860
view only (repetition removed): 6328


In [7]:
print(click_only_id.dtype) 
print(type(click_only_id))
overlap_lst = []
unoverlap_lst = []
for i in range(len(list(click_only_id))): 
    if list(click_only_id)[i] in list(view_only_id):
        #print('overlap id:',list(click_only_id)[i])
        overlap_lst.append(list(click_only_id)[i])
    else:
        unoverlap_1st.append(list(click_only_id)[i])
        
print('overlap list size (click & view):',np.size(overlap_lst))
print('non-overlap list size (click w/o view):',np.size(unoverlap_lst))
#print(overlap_lst)

int64
<class 'pandas.core.series.Series'>
overlap list size (click & view): 1860
non-overlap list size (click w/o view): 0


In [8]:
print(view_only_id.dtype) 
print(type(view_only_id))
overlap_lst2 = []
unoverlap_lst2 = []
for i in range(len(list(view_only_id))): 
    if list(view_only_id)[i] in list(click_only_id):
        #print('overlap id:',list(view_only_id)[i])
        overlap_lst2.append(list(view_only_id)[i])
    else:
        unoverlap_lst2.append(list(view_only_id)[i])
        
print('overlap list size (view & click):',np.size(overlap_lst2))
print('non-overlap list size (view w/o click):',np.size(unoverlap_lst2))
#print(overlap_lst2)

int64
<class 'pandas.core.series.Series'>
overlap list size (view & click): 1860
non-overlap list size (view w/o click): 4468


In [24]:
control_id = df[(df.group == 'control') & ~(df.group == 'experiment')].id
experimental_id = df[~(df.group == 'control') & (df.group == 'experiment')].id
control_experimental_id = df[(df.group == 'control') & (df.group == 'experiment')].id
print('control only:',np.size(control_id))
print('experimental only:',np.size(experimental_id))
print('both control & experimental:',np.size(control_experimental_id))
print('total:',np.size(control_id) + np.size(experimental_id) - np.size(control_experimental_id))

control only: 4264
experimental only: 3924
both control & experimental: 0
total: 8188


In [10]:
df['count'] = 1
df.head(20)

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1
5,2016-09-24 20:32:25.712659,681598,experiment,view,1
6,2016-09-24 20:39:03.248853,522116,experiment,view,1
7,2016-09-24 20:57:20.336757,349125,experiment,view,1
8,2016-09-24 20:58:01.948663,349125,experiment,click,1
9,2016-09-24 21:00:12.278374,560027,control,view,1


In [11]:
control_table = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control_table

action,click,view
id,,
182994,1.0,1.0
183089,NaN,1.0
183248,1.0,1.0
183515,NaN,1.0
183524,NaN,1.0
...,...,...
936786,NaN,1.0
937003,NaN,1.0
937073,NaN,1.0


In [12]:
control_table = control_table.fillna(0)
control_table

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


In [13]:
print("Sample sizes:\tControl: {}".format(np.size(control_table)))
print("Total Clicks:\tControl: {}".format(np.sum(control_table.click)))
print("Average click rate:\tControl: {}".format(np.mean(control_table.click)))

Sample sizes:	Control: 6664
Total Clicks:	Control: 932.0
Average click rate:	Control: 0.2797118847539016


In [14]:
experimental_table = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experimental_table

action,click,view
id,,
182988,NaN,1.0
183136,NaN,1.0
183141,1.0,1.0
183283,NaN,1.0
183389,NaN,1.0
...,...,...
935382,NaN,1.0
935576,NaN,1.0
935742,1.0,1.0


In [15]:
experimental_table = experimental_table.fillna(0)
experimental_table

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0
...,...,...
935382,0.0,1.0
935576,0.0,1.0
935742,1.0,1.0


In [16]:
print("Sample sizes:\tExperiment: {}".format(np.size(experimental_table)))
print("Total Clicks:\tExperiment: {}".format(np.sum(experimental_table.click)))
print("Average click rate:\tExperiment: {}".format(np.mean(experimental_table.click)))

Sample sizes:	Experiment: 5992
Total Clicks:	Experiment: 928.0
Average click rate:	Experiment: 0.3097463284379172


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [17]:
#Your code here
p_value_welch_ttest(control_table.click, experimental_table.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [18]:
#Your code here
control_rate = np.mean(control_table.click)
expected_experiment_clicks_under_null = control_rate * experimental_table.shape[0]
print(round(expected_experiment_clicks_under_null,3))

838.017


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [19]:
#Your code here
std = np.sqrt(experimental_table.shape[0] * control_rate * (1 - control_rate))
std

24.568547907005815

In [20]:
z = (np.sum(experimental_table.click) - expected_experiment_clicks_under_null) / std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [21]:
#Your code here
p = stats.norm.sf(z) #or 1 - stats.norm.cdf(z_score)
p

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [22]:
# Comment: Yes, while the p-value is slightly lower, 
# both would lead to confident rejection of the null hypothesis. 
# The experimental page appears to be a more effective design.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.